In [1]:
import os
import shutil
import numpy as np
import pickle as pk
import pandas as pd
from keras.utils import to_categorical ,Sequence
from keras import losses, models, optimizers
from keras.models import load_model
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from keras.layers import Activation, LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from sklearn.model_selection import KFold
from random_eraser import get_random_eraser
from keras.optimizers import Adam
from os.path import join
import resnet

map_dict = pk.load(open('data/map.pkl' , 'rb'))

Using TensorFlow backend.
/home/mike/Desktop/venv/lib/python3.5/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.23) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
semi = pd.read_csv('data/cotrain/leo_mfcc6_resnet_cotrain_Y.csv')
semi_name = semi['fname'].values

# unverified df
unverified_df = pd.read_csv('data/train_label.csv')
# test df 
test_df = pd.read_csv('data/sample_submission.csv')

#  append two df
unverified_df = unverified_df[unverified_df['fname'].isin(semi_name)]
unverified_df = unverified_df.drop(columns=['manually_verified'])

test_df = test_df[test_df['fname'].isin(semi_name)]

unverified_idx = unverified_df.index.values
test_idx = test_df.index.values

df = pd.concat([unverified_df , test_df])
df = df.drop(columns=['label'])
df['label_verified'] = semi['label_verified'].values
df['onehot'] = df['label_verified'].apply(lambda x: to_categorical(x,num_classes=41))


In [3]:
semi.tail(3)

,fname,label_verified
3606,ffbc5fb8.wav,4
3607,ffd137a4.wav,38
3608,fff6d073.wav,21


In [4]:
df.tail(3)

,fname,label_verified,onehot
9386,ffbc5fb8.wav,4,"[[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,..."
9389,ffd137a4.wav,38,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
9398,fff6d073.wav,21,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [5]:
X_unverified = np.load('data/mfcc/X_train.npy')[unverified_idx]
X_test = np.load('data/X_test.npy')[test_idx]

X_semi = np.append(X_unverified,X_test , axis=0)
Y_semi = np.array(df['onehot'].tolist()).reshape(-1,41)


print(X_semi.shape)
print(Y_semi.shape)

(3609, 40, 345, 1)
(3609, 41)


In [6]:
 # data generator ====================================================================================
class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        self.X_train = X_train
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y


# Training Semi Data

In [7]:

model_path = 'model_full_resnet152'
refine_path = 'model_full_resnet152_refine'

all_x = np.concatenate( (np.load('data/mfcc/X_train.npy') , np.load('data/X_test.npy')))

if not os.path.exists(refine_path):
    os.mkdir(refine_path)

for i in range(1,11):
    X_train = np.load('data/ten_fold_data/X_train_{}.npy'.format(i)) 
    Y_train = np.load('data/ten_fold_data/Y_train_{}.npy'.format(i)) 
    X_test = np.load('data/ten_fold_data/X_valid_{}.npy'.format(i))
    Y_test = np.load('data/ten_fold_data/Y_valid_{}.npy'.format(i))
    
    X_train = np.append(X_train,X_semi , axis=0)
    Y_train = np.append(Y_train,Y_semi , axis=0)
    
    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    
    model = load_model(join(model_path,'best_{}.h5'.format(i)))
    
    checkpoint = ModelCheckpoint(join(refine_path , 'semi_%d_{val_acc:.5f}.h5'%i), monitor='val_acc', verbose=1, save_best_only=True)
    early = EarlyStopping(monitor="val_acc", mode="max", patience=30)
    callbacks_list = [checkpoint, early]
    
    datagen = ImageDataGenerator(
        featurewise_center=True,  # set input mean to 0 over the dataset
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        preprocessing_function=get_random_eraser(v_l=np.min(all_x), v_h=np.max(all_x)) # Trainset's boundaries.
    )
    
    mygenerator = MixupGenerator(X_train, Y_train, alpha=1.0, batch_size=128, datagen=datagen)
    
    model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.0001),
             metrics=['accuracy'])
    # mixup
#     history = model.fit_generator(mygenerator(),
#                     steps_per_epoch= X_train.shape[0] // 128,
#                     epochs=10000,
#                     validation_data=(X_test,Y_test), callbacks=callbacks_list)
    # normalize
    history = model.fit(X_train, Y_train, validation_data=(X_test, Y_test), callbacks=callbacks_list,
                        batch_size=32, epochs=10000)

    
#     break

(6948, 40, 345, 1)
(6948, 41)
(371, 40, 345, 1)
(371, 41)
Train on 6948 samples, validate on 371 samples
Epoch 1/10000
6948/6948 [==============================] - 395s - loss: 0.2690 - acc: 0.9925 - val_loss: 1.3928 - val_acc: 0.8221
Epoch 2/10000
6948/6948 [==============================] - 160s - loss: 0.2464 - acc: 0.9978 - val_loss: 1.4326 - val_acc: 0.8248
Epoch 3/10000
6948/6948 [==============================] - 160s - loss: 0.2394 - acc: 0.9983 - val_loss: 1.3918 - val_acc: 0.8302
Epoch 4/10000
6948/6948 [==============================] - 150s - loss: 0.2351 - acc: 0.9991 - val_loss: 1.3535 - val_acc: 0.8221
Epoch 5/10000
6948/6948 [==============================] - 140s - loss: 0.2306 - acc: 0.9993 - val_loss: 1.3783 - val_acc: 0.8167
Epoch 6/10000
6948/6948 [==============================] - 140s - loss: 0.2286 - acc: 0.9993 - val_loss: 1.3398 - val_acc: 0.8275
Epoch 7/10000
6948/6948 [==============================] - 140s - loss: 0.2248 - acc: 0.9990 - val_loss: 1.3636 - v

6948/6948 [==============================] - 140s - loss: 0.1719 - acc: 0.9999 - val_loss: 1.3978 - val_acc: 0.8113
Epoch 33/10000
6948/6948 [==============================] - 140s - loss: 0.1695 - acc: 1.0000 - val_loss: 1.3900 - val_acc: 0.8032
Epoch 34/10000
6948/6948 [==============================] - 140s - loss: 0.1671 - acc: 1.0000 - val_loss: 1.3821 - val_acc: 0.8059
(6948, 40, 345, 1)
(6948, 41)
(371, 40, 345, 1)
(371, 41)
Train on 6948 samples, validate on 371 samples
Epoch 1/10000


ResourceExhaustedError: OOM when allocating tensor with shape[32,3,22,1024]
	 [[Node: training_3/Adam/gradients/zeros_169 = Fill[T=DT_FLOAT, _class=["loc:@add_86/add"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_3/Adam/gradients/Shape_170, training_3/Adam/gradients/zeros_169/Const)]]
	 [[Node: loss_3/add_154/_31335 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_73481_loss_3/add_154", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'training_3/Adam/gradients/zeros_169', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/mike/Desktop/venv/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
   